<a href="https://colab.research.google.com/github/chewzzz1014/csc4700-embedded-collision-detection/blob/master/src/train_collision_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
import joblib

In [17]:
csv_path_no_collision = "is_collision = 0 data.csv"
csv_path_collision = "is_collision = 1 data.csv"

df_no_collision = pd.read_csv(csv_path_no_collision)
df_collision = pd.read_csv(csv_path_collision)
df = pd.concat([df_no_collision, df_collision], ignore_index=True)

In [18]:
df.info()
print('********************************************')
df[df['is_collision'] == 0].info()
print('********************************************')
df[df['is_collision'] == 1].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 458 entries, 0 to 457
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   created_at    458 non-null    object 
 1   entry_id      458 non-null    int64  
 2   x             458 non-null    float64
 3   y             458 non-null    float64
 4   z             458 non-null    float64
 5   is_collision  458 non-null    int64  
 6   batch_id      458 non-null    int64  
dtypes: float64(3), int64(3), object(1)
memory usage: 25.2+ KB
********************************************
<class 'pandas.core.frame.DataFrame'>
Index: 184 entries, 0 to 437
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   created_at    184 non-null    object 
 1   entry_id      184 non-null    int64  
 2   x             184 non-null    float64
 3   y             184 non-null    float64
 4   z             184 non-null    float64
 

In [22]:
# Feature engineering: Extract statistical features for each batch_id
def extract_features(group):
    features = {
        "x_mean": group["x"].mean(),
        "x_std": group["x"].std(),
        "x_min": group["x"].min(),
        "x_max": group["x"].max(),
        "y_mean": group["y"].mean(),
        "y_std": group["y"].std(),
        "y_min": group["y"].min(),
        "y_max": group["y"].max(),
        "z_mean": group["z"].mean(),
        "z_std": group["z"].std(),
        "z_min": group["z"].min(),
        "z_max": group["z"].max(),
    }
    return pd.Series(features)

# Apply feature extraction
features = df.groupby("batch_id").apply(extract_features).reset_index()
labels = df.groupby("batch_id")["is_collision"].first().reset_index(name="is_collision")

<ipython-input-22-e94268fb9689>:20: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  features = df.groupby("batch_id").apply(extract_features).reset_index()


In [24]:
features.info()
print('********************************************')
labels.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 13 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   batch_id  100 non-null    int64  
 1   x_mean    100 non-null    float64
 2   x_std     97 non-null     float64
 3   x_min     100 non-null    float64
 4   x_max     100 non-null    float64
 5   y_mean    100 non-null    float64
 6   y_std     97 non-null     float64
 7   y_min     100 non-null    float64
 8   y_max     100 non-null    float64
 9   z_mean    100 non-null    float64
 10  z_std     97 non-null     float64
 11  z_min     100 non-null    float64
 12  z_max     100 non-null    float64
dtypes: float64(12), int64(1)
memory usage: 10.3 KB
********************************************
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype
---  ------        --------------  -----
 0   batch_id      100 non-nu

In [20]:
data = pd.merge(features, labels, on="batch_id")

# Split data into train and test sets
X = data.drop(columns=["batch_id", "is_collision"])
y = data["is_collision"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [21]:
X_train.info()
print('********************************************')
X_test.info()
print('********************************************')
y_train.info()
print('********************************************')
y_test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 80 entries, 55 to 51
Data columns (total 12 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   x_mean  80 non-null     float64
 1   x_std   77 non-null     float64
 2   x_min   80 non-null     float64
 3   x_max   80 non-null     float64
 4   y_mean  80 non-null     float64
 5   y_std   77 non-null     float64
 6   y_min   80 non-null     float64
 7   y_max   80 non-null     float64
 8   z_mean  80 non-null     float64
 9   z_std   77 non-null     float64
 10  z_min   80 non-null     float64
 11  z_max   80 non-null     float64
dtypes: float64(12)
memory usage: 8.1 KB
********************************************
<class 'pandas.core.frame.DataFrame'>
Index: 20 entries, 83 to 31
Data columns (total 12 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   x_mean  20 non-null     float64
 1   x_std   20 non-null     float64
 2   x_min   20 non-null     float64
 3   x_max   

In [16]:
X_train.describe()

,x_mean,x_std,x_min,x_max,y_mean,y_std,y_min,y_max,z_mean,z_std,z_min,z_max
count,80.0000,77.000000,80.000000,80.000000,80.000000,77.000000,80.000000,80.00000,80.000000,77.000000,80.000000,80.000000
mean,-1.8617,1.389548,-3.096375,-0.029000,0.819344,1.649193,-0.261625,3.35200,-8.528383,1.820206,-10.467875,-6.463000
std,1.2307,2.076526,2.124145,3.933532,1.597452,2.674015,1.587937,5.94723,2.651226,3.272139,5.675655,5.942035
min,-6.8800,0.017889,-11.530000,-6.320000,-7.310000,0.000000,-9.730000,-2.98000,-16.058000,0.000000,-48.640000,-9.260000
25%,-2.2795,0.144845,-3.510000,-1.800000,0.478500,0.181934,-0.530000,0.70000,-9.236000,0.105830,-9.917500,-9.020000
50%,-1.9380,0.595122,-2.390000,-1.350000,0.671000,0.639273,0.235000,1.08000,-9.036000,0.434143,-9.280000,-8.710000
75%,-1.4875,1.679077,-1.950000,0.010000,1.175500,1.865819,0.630000,2.16750,-8.700500,2.176051,-9.020000,-7.800000
max,3.4920,11.073058,0.550000,23.300000,6.482000,13.770640,0.860000,31.11000,6.303333,18.214143,5.450000,26.910000


In [7]:
clf = RandomForestClassifier(random_state=42)
clf.fit(X_train, y_train)

# Export trained model
joblib.dump(clf, "collision_clf_model.pkl")

['collision_clf_model.pkl']

In [8]:
y_pred = clf.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.85
Classification Report:
               precision    recall  f1-score   support

           0       1.00      0.62      0.77         8
           1       0.80      1.00      0.89        12

    accuracy                           0.85        20
   macro avg       0.90      0.81      0.83        20
weighted avg       0.88      0.85      0.84        20



In [9]:
new_batch = pd.DataFrame({
    "x": [-2.04, -2.12, -1.96, -2.08, -2.24],
    "y": [0.75, 0.86, 0.67, 0.39, 0.51],
    "z": [-9.06, -9.34, -8.98, -8.90, -9.02]
})
new_features = extract_features(new_batch)
print("Prediction for new batch:", clf.predict([new_features]))

Prediction for new batch: [0]


/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
